# Nowcasting PDIR data

This notebook provide instructions on how to nowcast using the data that was previously downloaded. This notebook will cover how to 

1. Use the dataloader from the h5 files to produce windowed inputs
2. Initialize config files for each nowcasting model
3. Nowcast the generated inputs and save the outputs in a format ready to be used for downstream tasks

Author: Akshay Aravamudan (aaravamudan2014@my.fit.edu)

Last Edited: Jan 21, 2025

## Imports

In [1]:
from servir.utils.evaluation import generate_outputs
from servir.core.data_provider import IMERGDataModule

Pysteps configuration file found at: /volume/NFS/aa3328/miniconda3/envs/tito_env/lib/python3.12/site-packages/pysteps/pystepsrc



/volume/NFS/aa3328/miniconda3/envs/tito_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Initialize parameters

In [2]:
# event id for which the data was downloaded
event_id = 'PDIR_data'

# location of the h5 file that was generated after downloading the data
h5_dataset_location = '../data/events/'+str(event_id)+'.h5'

# as of now, we do not have IR data, so we set it None
ir_h5_dataset_location = None

# this string is used to determine the kind of dataloader we need to use
# for processing individual events, we reccommend the user to keep this fixed
dataset_type = 'wa'

## Initialize the dataloader

In [4]:
data_provider =  IMERGDataModule(
        forecast_steps = 12,
        history_steps = 8,
        imerg_filename = h5_dataset_location,
        ir_filename = ir_h5_dataset_location,
        batch_size = 32,
        image_shape = (360, 516),
        normalize_data=False,
        dataset = dataset_type,
        production_mode = True)

# for now we are treating the test dataloader as the main one since we are only interested in predicting for individual events
data_loader = data_provider.test_dataloader()

original shape (9, 360, 516)
Precipitation Dataset input shape:  (0, 8, 1, 360, 516)


## Initialize config files for individual models and predict

For producing teh nowcasts using each of the models, the following parameters need to be initialized

1. model_type: name of the model ('convlstm', 'linda', 'steps', 'lagrangian', 'naive')
2. model_config_location: location where the python file with model specific parameters can be found. These have already been populated in the `configs/wa_imerg/` directory.
3. model_save_location: if the model requires a saved object (for instance neural network weights), this parameter points to that location.
4. use_gpu: flag for whether the model should use a GPU. note that this is only applicable for neural network models and the node which runs this notebook must contain a GPU.


### ConvLSTM

In [5]:
from servir.utils.m_h5py2tif import h5py2tif
from servir.core.model_picker import ModelPicker

model_type = 'convlstm'
model_config_location = 'configs/wa_imerg/ConvLSTM.py'
model_save_location = 'temp/imerg_only_mse_params.pth'
model_output_location = 'results/'
use_gpu = False
produce_ensemble_outputs = False
convlstm_output = generate_outputs(data_loader, model_type, model_config_location, model_save_location, use_gpu, produce_ensemble_outputs=produce_ensemble_outputs, event_name = event_id, model_output_location=model_output_location)

model_picker = ModelPicker(model_type=model_type, 
                            model_config_location=model_config_location, 
                            model_save_location=model_save_location,
                            use_gpu=use_gpu)
model_picker.load_data('../data/events/'+str(event_id)+'.h5')

model_picker.load_model(get_ensemble=False)

# predictions = model_picker.predict()

model_picker.save_output(str(event_id)+'_outputs.h5', convlstm_output, num_predictions=len(convlstm_output))
# h5py2tif(h5_fname=str(event_id)+'_outputs.h5', 
#          meta_fname = '/vol_efthymios/NFS07/en279/SERVIR/TITO_test3/ML/nowcasting/data/events/1/metadata.json', 
#          tif_directory = '/vol_efthymios/NFS07/en279/SERVIR/TITO_test3/ML/nowcasting/data/events/1/predictions/',
#          num_predictions=len(convlstm_output))


loading config from configs/wa_imerg/ConvLSTM.py ...
loading config from configs/wa_imerg/ConvLSTM.py ...


### STEPS

In [6]:
from servir.utils.m_h5py2tif import h5py2tif
from servir.core.model_picker import ModelPicker

model_type = 'steps'
model_config_location = 'configs/wa_imerg/PySTEPS.py'
model_save_location = ''
use_gpu = False
produce_ensemble_outputs = False
model_output_location = 'results/'
steps_output = generate_outputs(data_loader, model_type, model_config_location, model_save_location, use_gpu, produce_ensemble_outputs=produce_ensemble_outputs, event_name = event_id,model_output_location = model_output_location)
# tif file per prediction

model_picker = ModelPicker(model_type=model_type, 
                            model_config_location=model_config_location, 
                            model_save_location=model_save_location,
                            use_gpu=use_gpu)
model_picker.load_data('../data/events/'+str(event_id)+'.h5')

model_picker.load_model(get_ensemble=False)

# predictions = model_picker.predict()

model_picker.save_output(str(event_id)+'_outputs.h5', steps_output, num_predictions=len(steps_output))
# h5py2tif(h5_fname=str(event_id)+'_outputs.h5', 
#          meta_fname = '/vol_efthymios/NFS07/en279/SERVIR/TITO_test3/ML/nowcasting/data/events/1/metadata.json', 
#          tif_directory = '/vol_efthymios/NFS07/en279/SERVIR/TITO_test3/ML/nowcasting/data/events/1/predictions/',
#          num_predictions=len(steps_output),
#          method='steps')

loading config from configs/wa_imerg/PySTEPS.py ...


loading config from configs/wa_imerg/PySTEPS.py ...


### Lagrangian Persistence

In [8]:
from servir.utils.m_h5py2tif import h5py2tif
from servir.core.model_picker import ModelPicker

model_type = 'lagrangian'
model_config_location = 'configs/wa_imerg/lagrangian_persistence.py'
model_save_location = ''
use_gpu = False
produce_ensemble_outputs = False
model_output_location = 'results/'
lagrangian_output = generate_outputs(data_loader, model_type, model_config_location, model_save_location, use_gpu, produce_ensemble_outputs=produce_ensemble_outputs, event_name = event_id,model_output_location = model_output_location)

model_picker = ModelPicker(model_type=model_type, 
                            model_config_location=model_config_location, 
                            model_save_location=model_save_location,
                            use_gpu=use_gpu)
model_picker.load_data('../data/events/'+str(event_id)+'.h5')

model_picker.load_model(get_ensemble=False)

lagrangian_output = model_picker.predict()

# lagrangian_output = lagrangian_output[None, :, :, :]
model_picker.save_output(str(event_id)+'_outputs.h5', lagrangian_output, num_predictions=len(lagrangian_output))

print(lagrangian_output.shape)
# h5py2tif(h5_fname=str(event_id)+'_outputs.h5', 
#          meta_fname = '/vol_efthymios/NFS07/en279/SERVIR/TITO_test3/ML/nowcasting/data/events/1/metadata.json', 
#          tif_directory = '/vol_efthymios/NFS07/en279/SERVIR/TITO_test3/ML/nowcasting/data/events/1/predictions/',
#          num_predictions=len(lagrangian_output))



loading config from configs/wa_imerg/lagrangian_persistence.py ...


loading config from configs/wa_imerg/lagrangian_persistence.py ...
(12, 900, 1295)


### Naive Persistence

In [9]:
from servir.utils.m_h5py2tif import h5py2tif
from servir.core.model_picker import ModelPicker

model_type = 'naive'
model_config_location = 'configs/wa_imerg/naive_persistence.py'
model_save_location = ''
use_gpu = False
produce_ensemble_outputs = False
model_output_location = 'results/'
naive_output = generate_outputs(data_loader, model_type, model_config_location, model_save_location, use_gpu, produce_ensemble_outputs=produce_ensemble_outputs, event_name = event_id,model_output_location = model_output_location)

model_picker = ModelPicker(model_type=model_type, 
                            model_config_location=model_config_location, 
                            model_save_location=model_save_location,
                            use_gpu=use_gpu)
model_picker.load_data('../data/events/'+str(event_id)+'.h5')

model_picker.load_model(get_ensemble=False)

model_picker.save_output(str(event_id)+'_outputs.h5', naive_output, num_predictions=len(naive_output))

# h5py2tif(h5_fname=str(event_id)+'_outputs.h5', 
#          meta_fname = '/vol_efthymios/NFS07/en279/SERVIR/TITO_test3/ML/nowcasting/data/events/1/metadata.json', 
#          tif_directory = '/vol_efthymios/NFS07/en279/SERVIR/TITO_test3/ML/nowcasting/data/events/1/predictions/',
#          num_predictions=len(naive_output))

loading config from configs/wa_imerg/naive_persistence.py ...


/volume/NFS/aa3328/miniconda3/envs/tito_env/lib/python3.12/site-packages/torch/utils/data/_utils/collate.py:285: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  return collate([torch.as_tensor(b) for b in batch], collate_fn_map=collate_fn_map)


starting predictions for batch 0
The shape of the resulting array is:  (12, 360, 516)
Advecting the radar rainfall fields took  0.008693218231201172  seconds
The shape of the resulting array is:  (12, 360, 516)
Advecting the radar rainfall fields took  0.002703428268432617  seconds
The shape of the resulting array is:  (12, 360, 516)
Advecting the radar rainfall fields took  0.0031270980834960938  seconds
The shape of the resulting array is:  (12, 360, 516)
Advecting the radar rainfall fields took  0.002671480178833008  seconds
The shape of the resulting array is:  (12, 360, 516)
Advecting the radar rainfall fields took  0.0027790069580078125  seconds
The shape of the resulting array is:  (12, 360, 516)
Advecting the radar rainfall fields took  0.0027272701263427734  seconds
The shape of the resulting array is:  (12, 360, 516)
Advecting the radar rainfall fields took  0.003042459487915039  seconds
loading config from configs/wa_imerg/naive_persistence.py ...


In [ ]:
predictions.shape

### LINDA

In [ ]:
model_type = 'linda'
model_config_location = 'configs/wa_imerg/LINDA.py'
model_save_location = ''
use_gpu = False
produce_ensemble_outputs = False
model_output_location = 'results/'
naive_output = generate_outputs(data_loader, model_type, model_config_location, model_save_location, use_gpu, produce_ensemble_outputs=produce_ensemble_outputs, event_name = event_id,model_output_location = model_output_location)
